configuration:
models_ver - insert YOLO version's numbers that the UAP will be trained on.

epsilon, lambda_1, lambda_2 - attack's parameters. more information can be found in the [paper](https://arxiv.org/abs/2205.13618)

BDD_IMG_DIR - a path to the BDD validation set images (or any other wanted dataset)

BDD_LAB_DIR - a path to the BDD validation set labels (or any other wanted dataset)

In [1]:
models_vers = [8] # for example: models_vers = [5] or models_vers = [3, 4, 5]
epsilon = 70
lambda_1 = 1
lambda_2 = 10
seed = 42
patch_size=(640,640)
img_size=(640,640)
batch_size = 8
num_workers = 4
max_labels_per_img = 65
BDD_IMG_DIR = '/Users/coenschoof/miniconda/envs/phantomsponges/BDD100K-to-YOLOV5/bdd_in_YOLOV5_train_newLabels/images/val'
BDD_LAB_DIR = '/Users/coenschoof/miniconda/envs/phantomsponges/BDD100K-to-YOLOV5/bdd_in_YOLOV5_train_newLabels/labels/val'

Load BDD dataset:

In [2]:
import torch
import os
import random
import numpy

from datasets.augmentations1 import train_transform
from datasets.split_data_set_combined import SplitDatasetCombined_BDD

/Users/coenschoof/miniconda/envs/phantomsponges/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# batch = [(1, 'a'), (2, 'b'), (3, 'c')]

# unzipped = zip(*batch)

# numbers, letters = unzipped

# print(numbers)  # Output: (1, 2, 3)
# print(letters)  # Output: ('a', 'b', 'c')

#dus ipv [(tensor, array, string), (tensor, array, string), (tensor, array, string)] hebben we
# train_loader[0] bevat 8 images
# train_loader[1] bevat alle labels per voor 8 images
# train_loader[2] bevat paths naar 8 images
def collate_fn(batch):
    return tuple(zip(*batch))

def set_random_seed(seed_value, use_cuda=True):
    numpy.random.seed(seed_value)  # cpu vars
    torch.manual_seed(seed_value)  # cpu  vars
    random.seed(seed_value)  # Python
    os.environ['PYTHONHASHSEED'] = str(seed_value)  # Python hash buildin
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  # gpu vars
        torch.backends.cudnn.deterministic = True  # needed
        torch.backends.cudnn.benchmark = False

In [4]:
split_dataset = SplitDatasetCombined_BDD(
            img_dir= BDD_IMG_DIR,
            lab_dir= BDD_LAB_DIR,
            max_lab=max_labels_per_img,
            img_size=img_size,
            transform=train_transform,
            collate_fn=collate_fn)

In [5]:
#train_loader en val_loader 

#train loader bevat een tuple van 3, elk element van de tuple bevat 8 samples (8 samples maakt 1 batch)
#De tuple bestaat uit het volgende:
    #[0] = de image met size (3,640,640)
    #[1] = de bounding boxes met size (x, 5) (5 bestaat uit class, x, y, x, y)
        #lijkt nergens gebruikt te worden
    #[2] = de naam van de image (e.g. 'c98258a4-54a47ff2.jpg')  

    #HEB IK AL DEZE INFORMATIE OOK NODIG WANNEER IK WIL TESTEN OP PASCAL/MTSD? converten is een pain
        #WORDT [2] ERGENS GEBRUIKT? IK WEET DAT [1] NERGENS GEBRUIKT WORDT
train_loader, val_loader, test_loader = split_dataset(val_split=0.1,
                                                      shuffle_dataset=True,
                                                      random_seed=seed,
                                                      batch_size=batch_size,
                                                      ordered=False,
                                                      collate_fn=collate_fn)

In [6]:
import numpy
from attack.uap_phantom_sponge_yolov8 import UAPPhantomSponge

torch.cuda.empty_cache()

patch_name = r"yolov"
for ver in models_vers:
  patch_name += f"_{ver}"
patch_name += f"_epsilon={epsilon}_lambda1={lambda_1}_lambda2={lambda_2}"

uap_phantom_sponge_attack = UAPPhantomSponge(patch_folder=patch_name, 
                                             train_loader=train_loader, 
                                             val_loader=val_loader, 
                                             epsilon = epsilon, 
                                             lambda_1=lambda_1, 
                                             lambda_2=lambda_2, 
                                             patch_size=patch_size, 
                                             models_vers=models_vers)
#adv_img = uap_phantom_sponge_attack.run_attack()

ModuleNotFoundError: No module named 'ultralytics'

In [7]:
pwd

'/Users/coenschoof/miniconda/envs/phantomsponges/PhantomSponges'

In [ ]:
pwd

'/Users/coenschoof/miniconda/envs/phantomsponges/PhantomSponges/local_yolos/yolov8'

# CODE FOR EVALUATION

In [ ]:
import torch
import torch.nn.functional as F
from PIL import Image
import numpy as np
from local_yolos.yolov5.models.experimental import attempt_load
from local_yolos.yolov5.utils.general import non_max_suppression
from torchvision import transforms
from local_yolos.yolov5.utils.metrics import box_iou
from local_yolos.yolov5.utils.plots import Annotator
import time
from tqdm import tqdm

In [ ]:
def perturb_img(img_name, change_aspect_ratio = True, output_type = "torch", patch_name = 'final_patch.png'):
    image_path = "/Users/coenschoof/miniconda/envs/phantomsponges/BDD100K-to-YOLOV5/bdd_in_YOLOV5_train_newLabels/images/val/" + img_name
    image = Image.open(image_path)
    image = image.resize((640, 640))
    transform = transforms.Compose([transforms.ToTensor()])
    clean_img = transform(image)

    # Load the image using PIL
    patch_path =  "/Users/coenschoof/miniconda/envs/phantomsponges/PhantomSponges/final_patch.png"
    patch = Image.open(patch_path)
    patch = transform(patch)

    perturbed_img = torch.clamp(clean_img + patch, 0, 1)

    # Assuming you have a torch tensor named torch_image
    # You can resize the tensor to (H, W, C) shape using transpose and multiply by 255 (if it's not already in the correct range)
    #pil_image = transforms.ToPILImage()(result.squeeze().cpu() * 255)
    tensor_min = torch.min(perturbed_img)
    tensor_max = torch.max(perturbed_img)
    normalized_tensor = (perturbed_img - tensor_min) / (tensor_max - tensor_min)

    # Convert the normalized tensor to a PIL image
    pil_image = transforms.ToPILImage()(normalized_tensor.squeeze().cpu())
    if change_aspect_ratio:
        random_scale_w = random.uniform(0.7, 0.9) #1.5, 3
        #random_scale_h = random.uniform(1, 2.5)
        width = int(640 * random_scale_w)
        width = (width // 32) * 32 #ensures that the new width is a multiple of 32, otherwise the model does not work
        height = 640 #height = int(640 * random_scale_h)
        height = (height // 32) * 32
        pil_image = pil_image.resize((width, height))

    if output_type == "torch":
        perturbed_image = transform(pil_image)
    elif output_type == "numpy":
        perturbed_image = np.array(pil_image)
    elif output == "pil":
        pass
    else:
        print("Unsupported output type provided!")

    return(perturbed_image)

In [ ]:
image_names = []
for image_np, _, image_name in tqdm(test_loader):
    image_names.append(image_name[0])

100%|██████████| 500/500 [00:07<00:00, 67.04it/s]


# New loss term experiments

In [ ]:
#create a tiny batch containing 3 images; 2 random images from the test set, and a fully white image

zeros_tensor = torch.zeros(1, 3, 640, 640)
#zero_output = model(zeros_tensor)[0]

random_index = random.randrange(len(image_names))
other_image = perturb_img(image_names[random_index], change_aspect_ratio = False, output_type = "torch").unsqueeze(0)
random_index = random.randrange(len(image_names))
other_image_2 = perturb_img(image_names[random_index], change_aspect_ratio = False, output_type = "torch").unsqueeze(0)
pil_image = transforms.ToPILImage()(other_image.squeeze())
#display(pil_image)

tiny_batch = torch.cat((other_image, other_image_2, zeros_tensor))
#tiny_batch_output = model(tiny_batch)

In [ ]:
cd local_yolos/yolov8

/Users/coenschoof/miniconda/envs/phantomsponges/PhantomSponges/local_yolos/yolov8


In [ ]:
pwd

'/Users/coenschoof/miniconda/envs/phantomsponges/PhantomSponges/local_yolos/yolov8'

In [ ]:
from local_yolos.yolov8.ultralytics.models.yolo.model import YOLO

# Load a model
model = YOLO("yolov8n.pt")

In [ ]:
results = model(tiny_batch)


0: 640x640 11 cars, 1: 640x640 4 cars, 2: 640x640 (no detections), 894.8ms
Speed: 0.0ms preprocess, 298.3ms inference, 5.5ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
results[1].boxes.data

tensor([[3.7095e+02, 2.4161e+02, 4.7751e+02, 3.8985e+02, 7.4596e-01, 2.0000e+00],
        [3.3962e+02, 2.4784e+02, 3.8285e+02, 3.2570e+02, 7.0045e-01, 2.0000e+00],
        [5.1293e+02, 2.7410e+02, 5.8143e+02, 3.4322e+02, 3.4601e-01, 2.0000e+00],
        [1.2336e+02, 6.2344e+02, 1.4905e+02, 6.3988e+02, 2.6008e-01, 2.0000e+00]])

In [ ]:
from attacks_tools.attack_utils import CustomPredictor

In [ ]:
#ONDERSTAANDE OUTPUT MOET NAAR DEZE SHAPE WORDEN OMGEZET

"""Runs Non-Maximum Suppression (NMS) on inference results

Returns:
        list of detections, on (n,6) tensor per image [xyxy, conf, cls]
"""

#output_patch.size() = torch.Size([8, 25200, 85])
    #8 plaatjes per batch
    #25200 bounding boxes per plaatje
    #85 = (x, y, width, height), object confidence score, and class probabilities. (dus 80 class probs)
#print(output_patch.size())

predictor = CustomPredictor(overrides = dict(model='yolov8n.pt') )
results = predictor(tiny_batch)

[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


In [ ]:
results[0][..., :4].shape

torch.Size([3, 84, 4])

In [ ]:
print(torch.swapaxes(results[0], 1, 2).shape)
torch.set_printoptions(sci_mode=False)
torch.swapaxes(results[0], 1, 2)[0, 0, :]

torch.Size([3, 8400, 84])


tensor([    -4.7234,     -7.9282,      4.8334,      7.8545,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,
             0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,
             0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,      0.0000,
             0.0000,      0.0000,      0

In [ ]:
from local_yolos.yolov8.ultralytics.utils.ops import non_max_suppression

In [ ]:
non_max_suppression(results[0])[1].shape

torch.Size([2, 6])

In [ ]:
pwd

'/Users/coenschoof/miniconda/envs/phantomsponges/PhantomSponges/local_yolos/yolov8'

In [ ]:
# self.task_map[self.task][key]

In [ ]:
model.task_map['detect']['predictor']()